In [24]:
from bs4 import BeautifulSoup as bs
import requests
import json
import os

In [25]:
items = [
			"2 seater sofa", "bench", "book cases",
			"coffee table", "dining set","queen beds",
			"arm chairs","chest drawers","garden seating",
			"bean bags","king beds"
		]

urls = []
dir_names = []

base_url = "https://www.pepperfry.com/site_product/search?q="

base_dir = "./pepperfry-data"
if not os.path.exists(base_dir):
  os.mkdir(base_dir)
base_path = "./pepperfry-data/"

for item in items:
  query_str = "+".join(item.split())
  dir_name = "-".join(item.split())

  urls.append(base_url+query_str)
  dir_path = base_path+dir_name
  dir_names.append(dir_path)
  if not os.path.exists(dir_path):
    os.mkdir(dir_path)

In [37]:
headers = {'User-Agent': 'Mozilla/5.0'}
for i in range(len(urls)):
  req = requests.get(urls[i], headers=headers)
  soup = bs(req.content)
  product_urls = soup.findAll('div', {'class': 'card-img-wrp center-xs card-srch-img-wrp'})
  # print(len(product_urls))
  for k in range(20):
    url = product_urls[k]
    product_page_url = url.a.attrs['href']
    product_page = requests.get(product_page_url, headers = headers)
    product_page_soup = bs(product_page.content)
    product_title = product_page_soup.find('h1', {'class':'v-pro-ttl pf-medium-bold-text'})
    product_dir = "-".join(product_title.text.split())
    dir_name = dir_names[i] + "/" + product_dir
    if not os.path.exists(dir_name):
      os.mkdir(dir_name)
    # print(product_title)
    product_img = product_page_soup.findAll('li', {'class':'vipImage__thumb-each noClickSlide'})
    for temp in range(len(product_img)): # enumerate to get the index
      if product_img[temp].find('a', {'class': 'vipImage__thumb-3d'}) == None:
        with open(dir_name +'/img{}.jpg'.format(temp), 'wb') as file:            
            img_url = product_img[temp].a.attrs["data-hoverimg"]
            response = requests.get(img_url)
            file.write(response.content)
    
    product_price = product_page_soup.findAll('div', {'itemprop': 'offers'})
    if len(product_price) >= 1:
      price = product_price[0].meta.attrs['content']
    else:
      price = product_page_soup.find('span', {'class': 'v-price-mrp-amt-only'})["data-price"]

    product_savings = product_page_soup.findAll('span', {'class': 'v-price-save-ttl-amt v-price-col-right total_saving'})
    savings = ""
    
    if len(savings) >= 1:
      savings = product_savings[0].text
    else:
      savings = "0"
    # print(savings)

    product_description = product_page_soup.find(['div', 'p'], {'class': 'v-more-info-tab-cont-para-wrap'})
    # for n in range(len(product_description)):
    #   description += product_description.findAll('p')[n].text
    # print(description)
    description = ""
    if len(product_description.findAll('p')) >= 1:
      description = product_description.findAll('p')[0].text
    else:
      description = product_description.text

    product_details = product_page_soup.findAll('div', {'class': 'v-prod-comp-dtls-listitem'})
    brand = product_details[0].findAll('span')[1].text
    dimensions = product_details[1].findAll('span')[1].text
    weight = product_details[2].findAll('span')[1].text
    warranty = product_details[3].findAll('span')[1].text
    assembly = product_details[4].findAll('span')[1].text
    fabric = product_details[5].findAll('span')[1].text


    d = {
        'price' : price,
        'savings': savings,
        'description': description,
        'details' : {
            'brand' : brand,
            'dimensions': dimensions,
            'weight': weight,
            'warranty': warranty,
            'assembly': assembly,
            'primary material': fabric
        }
    }

    with open(os.path.join(dir_name,'metadata.txt'),'w') as f:
        json.dump(d,f)

48
48
48
47
46
47
48
47
48
48
48
